In [38]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline


In [39]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese",mirror='https://mirrors.tuna.tsinghua.edu.cn/hugging-face-models')
model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese",mirror='https://mirrors.tuna.tsinghua.edu.cn/hugging-face-models')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
text = "中国在[MASK]洲."
inputs = tokenizer(text, return_tensors="pt")
inputs = tokenizer(text, return_tensors="pt")
print(tokenizer.mask_token_id)
print(inputs)

103
{'input_ids': tensor([[ 101,  704, 1744, 1762,  103, 3828,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [41]:
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
logits = model(**inputs).logits
mask_token_logits = logits[0, mask_token_index, :]
mask_token_logits

tensor([[-7.7195, -8.3129, -8.2264,  ..., -6.7415, -6.8784, -6.1717]],
       grad_fn=<IndexBackward0>)

In [42]:
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()

for token in top_3_tokens:
    print(text.replace(tokenizer.mask_token, tokenizer.decode([token])))

中国在欧洲.
中国在亚洲.
中国在非洲.


In [45]:
classifier = pipeline("zero-shot-classification",model="bert-base-chinese")
sequence_to_classify = "我要吃回锅肉"
candidate_labels = ['食物', '科技', '篮球']
classifier(sequence_to_classify, candidate_labels)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

{'sequence': '我要吃回锅肉',
 'labels': ['篮球', '食物', '科技'],
 'scores': [0.3482101559638977, 0.3296661674976349, 0.3221237361431122]}